<a href="https://colab.research.google.com/github/lucario6607/lc0fish/blob/master/Another_copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!sudo apt-get install ninja-build

!pip3 install meson

!git clone --branch rpe-cuda https://github.com/almaudoh/lc0.git lc0-pr

!cd lc0-pr && rm -rf build

!cd lc0-pr && meson build --buildtype release -Db_lto=true -Dgtest=false -Dblas=false -Dopencl=false -Dcudnn=false

!cd lc0-pr/build && ninja

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  ninja-build
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 111 kB of archives.
After this operation, 358 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ninja-build amd64 1.10.1-1 [111 kB]
Fetched 111 kB in 0s (1,353 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package ninja-build.
(Reading database ... 123630 files and director

In [3]:
!wget https://github.com/CallOn84/LeelaNets/raw/refs/heads/main/Nets/Elite%20Leela/EliteLeela.pb.gz

--2024-11-30 15:46:21--  https://github.com/CallOn84/LeelaNets/raw/refs/heads/main/Nets/Elite%20Leela/EliteLeela.pb.gz
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CallOn84/LeelaNets/refs/heads/main/Nets/Elite%20Leela/EliteLeela.pb.gz [following]
--2024-11-30 15:46:22--  https://raw.githubusercontent.com/CallOn84/LeelaNets/refs/heads/main/Nets/Elite%20Leela/EliteLeela.pb.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6551855 (6.2M) [application/octet-stream]
Saving to: ‘EliteLeela.pb.gz’

EliteLeela.pb.gz    100%[===================>]   6.25M  --.-KB/s    in 0.04s   

2024-11-30 15:46:22 

In [4]:
pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148497 sha256=4bde9f577cded4c0c1d0aa3d675ab158ed42350798cffb59578863444a006c0c
  Stored in directory: /root/.cache/pip/wheels/2e/2d/23/1bfc95db984ed3ecbf6764167dc7526d0ab521cf9a9852544e
Successfully built chess


In [6]:
import json
import os
import sys
import argparse
import chess
import chess.engine
import chess.pgn
import datetime

config_file = "config_v1.json"

if not os.path.exists(config_file):
    print(f"Configuration file {config_file} does not exist.")
    sys.exit(1)

with open(config_file, 'r') as f:
    config = json.load(f)

# Extract variables from config
lc0_path = config['lc0_path']
white_net = config['white_net']
black_net = config['black_net']
num_games = config['num_games']
pgn_file_name = config['pgn_file_name']
use_opening_book = config['use_opening_book']
opening_book = config.get('opening_book', '')
use_dropbox_upload = config['use_dropbox_upload']

engine1_options = config['engine1_options']
engine2_options = config['engine2_options']

depth_limit_w_nodes = config['depth_limit_w']
depth_limit_b_nodes = config['depth_limit_b']

# Handle Dropbox setup if needed
if use_dropbox_upload:
    import dropbox
    dropbox_params = config['dropbox_params']
    DROPBOX_ACCESS_TOKEN = dropbox_params.get('access_token')
    app_key = dropbox_params.get('app_key')
    app_secret = dropbox_params.get('app_secret')
    oauth2_refresh_token = dropbox_params.get('oauth2_refresh_token')

    dbx = dropbox.Dropbox(
        app_key=app_key,
        app_secret=app_secret,
        oauth2_refresh_token=oauth2_refresh_token
    )

# Set up the opening book or starting FENs
if use_opening_book:
    if not os.path.exists(opening_book):
        print(f"Opening book file {opening_book} does not exist.")
        sys.exit(1)
    with open(opening_book, 'r') as f:
        fens = [line.strip() for line in f if line.strip()]
else:
    fixed_start_fen = config.get('fixed_start_fen', chess.STARTING_FEN)
    fens = [fixed_start_fen]

# Replace 'white_net' and 'black_net' placeholders in engine options
def replace_net_paths(options):
    for key, value in options.items():
        if value == 'white_net':
            options[key] = white_net
        elif value == 'black_net':
            options[key] = black_net

replace_net_paths(engine1_options)
replace_net_paths(engine2_options)

# Initialize engines
engine1 = chess.engine.SimpleEngine.popen_uci(lc0_path)
engine2 = chess.engine.SimpleEngine.popen_uci(lc0_path)

# Configure engines
engine1.configure(engine1_options)
engine2.configure(engine2_options)

# Set node limits
depth_limit_w = chess.engine.Limit(nodes=depth_limit_w_nodes)
depth_limit_b = chess.engine.Limit(nodes=depth_limit_b_nodes)

# Open a PGN file to save the games
pgn_file = open(pgn_file_name, 'a')

# Load last saved progress, if available
if os.path.exists("progress.json"):
    with open("progress.json", "r") as f:
        progress_data = json.load(f)
        start_game = progress_data.get("game_number", 1)
else:
    start_game = 1

wins = 0
losses = 0
draws = 0

try:
    for game_number in range(start_game, num_games + 1):
        # Determine which FEN to use
        fen_index = (game_number - 1) % len(fens)
        start_fen = fens[fen_index]

        board = chess.Board(fen=start_fen)
        game = chess.pgn.Game()
        game.setup(board)
        node = game

        # Set game headers
        game.headers["Event"] = "Leela Queen Odds Training"
        game.headers["Site"] = "Local"
        game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d")
        game.headers["Round"] = str(game_number)
        game.headers["White"] = "LeelaZero_White"
        game.headers["Black"] = "LeelaZero_Black"

        try:
            while not board.is_game_over():
                if board.turn == chess.WHITE:
                    # White's turn
                    result = engine1.play(board, depth_limit_w)
                else:
                    # Black's turn
                    result = engine2.play(board, depth_limit_b)

                board.push(result.move)
                node = node.add_variation(result.move)
        except Exception as e:
            print(f"Exception during game {game_number}: {e}")
            # If an error occurs, skip to the next game
            continue

        # Game over, set the result
        game.headers["Result"] = board.result()

        match board.result():
            case '1-0':
                wins += 1
            case '0-1':
                losses += 1
            case _:
                draws += 1

        # Calculate total games
        total_games = wins + draws + losses

        # Save the game to the PGN file
        print(game, file=pgn_file, end="\n\n")

        # Upload the PGN file to Dropbox every 100 games
        if use_dropbox_upload and total_games % 100 == 0:
            # Close the PGN file to ensure all data is written
            pgn_file.close()

            try:
                # Dropbox upload code
                with open(pgn_file_name, "rb") as f:
                    dropbox_path = f"/{pgn_file_name}"
                    dbx.files_upload(f.read(), dropbox_path, mode=dropbox.files.WriteMode.overwrite)
                    print(f"Uploaded {pgn_file_name} to Dropbox.")
            except dropbox.exceptions.ApiError as e:
                print(f"Failed to upload to Dropbox: {e}")

            # Reopen the PGN file in append mode for further writing
            pgn_file = open(pgn_file_name, "a")

        # Calculate total points earned
        total_points = wins + 0.5 * draws

        # Calculate scoring percentage
        scoring_percentage = (total_points / total_games) * 100
        print(f"Total Games: {total_games}")
        print(f"{wins} Wins, {draws} Draws, {losses} Losses")
        print("Scoring Percentage: {:.2f}%".format(scoring_percentage))

        # Update progress
        with open("progress.json", "w") as f:
            json.dump({"game_number": game_number + 1}, f)

finally:
    # Quit engines
    engine1.quit()
    engine2.quit()
    # Close PGN file
    pgn_file.close()


Total Games: 1
0 Wins, 0 Draws, 1 Losses
Scoring Percentage: 0.00%
Total Games: 2
0 Wins, 0 Draws, 2 Losses
Scoring Percentage: 0.00%
Total Games: 3
0 Wins, 0 Draws, 3 Losses
Scoring Percentage: 0.00%
Total Games: 4
0 Wins, 0 Draws, 4 Losses
Scoring Percentage: 0.00%
Total Games: 5
0 Wins, 0 Draws, 5 Losses
Scoring Percentage: 0.00%
Total Games: 6
0 Wins, 0 Draws, 6 Losses
Scoring Percentage: 0.00%


EngineTerminatedError: engine event loop dead

In [ ]:
/root/.cache/lc0/data-pihtuobzissm